In [1]:
import pandas as pd
import numpy as np

### Estimating the number of cases in a region of interest

The most interesting quantity for the mission of Opendemic is the number of cases in a Region Of Interest (ROI). In particular, this region will be a disk centered at the user's coordinates with a fixed radius. If one day we will have data with high spatial resolution, the disk will become a bad approximation fairly quickly. For this reason, the analysis that will be presented in the next paragraphs works for a general definition of ROI which just needs to be $L_1$ measurable, namely we want to be able to know its surface.

### Data
In this example we will consider the case of a user located in New York City (a.k.a. *Region*) and we will use the data available at [this link](https://www.vox.com/2020/3/26/21193848/coronavirus-us-cases-deaths-tests-by-state), which claim to be provided by *COVID Tracking Project, Census Bureau* and to be updated at March 30.
I don't know how reliable they are, but they will still be useful for our proof of concept. As soon as we have more reliable data we can plug them in the model.

We will assume to know the surface and the population density of NYC.

In [16]:
positive_test = 59513
total_test = 172360
empirical_infected_rate = positive_test / total_test

In [2]:
# https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population_density
a = pd.read_csv("states_density.csv")
a.head(2)

,State,Population density,Population density permi2,Population density perkm2,Population,Population.1,Land area,Land area mi2,Land area km2
0,District of Columbia,1,11011,4251,50,"672,228",56,61,158
1,New Jersey,2,1218,470,11,"8,958,013",46,"7,354","19,046.80"


In [3]:
# testing data by state
# https://covidtracking.com/ 
b = pd.read_csv("/Users/genecentrix/Desktop/Opendemic/states.csv")
b.head(2)

,name,state,positive,positiveScore,negativeScore,negativeRegularScore,commercialScore,grade,score,negative,...,death,total,lastUpdateEt,checkTimeEt,totalTestResults,fips,dateModified,dateChecked,notes,hash
0,Alaska,AK,119,1,1,1,1,A,4,3594,...,3,3713,3/30/20 19:00,3/30/20 22:42,3713,2,2020-03-30T23:00:00Z,2020-03-31T02:42:00Z,"Please stop using the ""total"" field. Use ""tota...",0ce775392949b669204417bf3b8fe3bcdc3e1084
1,Alabama,AL,947,1,1,0,1,B,3,5694,...,6,6641,3/30/20 0:00,3/30/20 22:42,6641,1,2020-03-30T04:00:00Z,2020-03-31T02:42:00Z,"Please stop using the ""total"" field. Use ""tota...",4eb0bcae98d9964b98cfb96af0b0ca042531a207


In [4]:
# https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population_density
c = pd.read_csv("cities_pop.csv", thousands=',')
c.head(2)

,Rank,Incorporated place,Metropolitan area,State,Population(2010 census),Land area(mi2),Land area(approximatekm2),Populationdensity(people per mi2),Populationdensity(approximatepeople per km2)
0,1,Guttenberg,New York City,New Jersey,11176,0.196,0.508,57116.0,22015.7
1,2,Union City,New York City,New Jersey,66455,1.280,3.320,51810.1,20045.6


In [5]:
state_df = a[['State','Land area km2', 'Population density perkm2']]
state_df.columns = ['state', 'area', 'density']
state_df.head(3)

,state,area,density
0,District of Columbia,158,4251
1,New Jersey,"19,046.80",470
2,Rhode Island,"2,678.00",394


In [6]:
city_df = c[['Incorporated place', 'State','Land area(approximatekm2)', 'Populationdensity(approximatepeople per km2)']]
city_df.columns = ['place', 'state','area', 'density']
city_df.head(5)

,place,state,area,density
0,Guttenberg,New Jersey,0.508,22015.7
1,Union City,New Jersey,3.320,20045.6
2,West New York,New Jersey,2.610,19059.0
3,Hoboken,New Jersey,3.320,15083.6
4,Kaser,New York,0.440,10729.1


In [7]:
city_df[city_df['place'] == 'New York City']

,place,state,area,density
5,New York City,New York,783.73,10431.1


In [13]:
# Data from https://en.wikipedia.org/wiki/New_York_City
region_surface = 783.84 * 1e6 # square meters
region_density = 10715 * 1e-6 # people per square meter

Now, we want to know the rate of infected people in the area.
The estimate can be as sophisticated as we want, but the first as-simple-as-wrong guess is that it is equal to the number of positive tests in the region divided by the total number of performed tests in the region.
This estimate is agnostic to all the asymptomatic non-tested infected cases and assumes that all symptomatic cases have been tested.

In [9]:
rates = []
for ind, i in b.iterrows():
    # empirical_infected_rate = positive_test / total_test
    rates.append(i['positive'] / i['total']) 
b['infected_rate'] = rates

In [10]:
b[['name','infected_rate']].head(4)

,name,infected_rate
0,Alaska,0.032050
1,Alabama,0.142599
2,Arkansas,0.084273
3,Arizona,0.069038


In [11]:
b[b['name'] == 'New York'][['state','infected_rate']]

,state,infected_rate
34,NY,0.356613


# Correcting the estimation of the volume of the infected population

If the rate of asymptomatic cases $r_a$ is known, we can correct the estimation of the infected rate with the following strategy.

[link to paper](https://drive.google.com/file/d/1qrd4GxxGTz-MoEk6PzzfBwbSPUxMQufd/view)


Consider the total number of infected people $I$ which is the sum of the symptomatic and asymptomatic cases $I_s$ and $I_a$ respectively.

\begin{equation*}
I = I_s + I_a
\label{eq:totalinfected}\tag{1}
\end{equation*}

The number of asymptomatic infected people will be $I_a = r_a \cdot I$.
We can then rewrite equation \eqref{eq:totalinfected} as

\begin{equation*}
I = I_s + r_a \cdot I
\end{equation*}

obtaining the following estimate of I.

\begin{equation*}
I = \frac{I_s}{1 - r}
\end{equation*}

### In practice ...
In practive, we are given the number of positive tests and the number of people in the population.
We don't know xactly how many of these positive tests correspond to symptomatic cases (the literature is very confused and confusing).
If we assume **all** of the positive tests to be symptomatic, we have an over-estimate that at least does the job.
Notice that this assumption is reasonable as far as testing is made accessible primarily to people self-reporting symptoms. See REF for a discussion about this.

Exploiting this idea, we can define the *correct infected rate* as a function of the *empirical infected rate* as follows.

In [14]:
def old_correct_infected_rate(eir, **kwargs):
    return eir

In [ ]:
def correct_infected_rate(eir, asymptomatic_rate=0, **kwargs):
    """
    Function that computes the corrected rate of infected people.    
    
    Args:
        eir: float empirical infected rate
        asymptomatic_rate: float Rate of infected people that do not show symptoms
    Returns:
        float Corrected infected rate
    """
    if  not (0 < asymptomatic_rate and asymptomatic_rate < 1):
        raise ValueError('The asymptomatic_rate variable must be between 0 and 1')
    return eir / (1 - asymptomatic_rate)

### Estimates of the asymptomatic rate
It is **very** difficult to estimate the rate of asymptomatic infected people among the infected population, basically because it is impossible to monitor everyone in continuous time.

The literature about is in strong disagreement. The most accurate available experiment is the one performed in the [Diamond Princess study](https://eurosurveillance.org/content/10.2807/1560-7917.ES.2020.25.10.2000180#abstract_content), where they report the rate to be $r_a = 0.17,9$. This study is biased by the age of the population, which is reported to be predominantly above 60 years old. As a consequence, the insurgence of symptoms is assumed to be more likely, therefore the asymptomatic rate could be lower in practice.
Over estimating the asymptomatic case makes the correct infected rate higher, which is not bad in our case since it is both a stay-at-the-course strategy that has the effect to increase the feeling of being surrounded by infected people.

In order to retrieve the number of *cases around me* (CAM), we need to know the number of *people around me* (PAM) and the *infected rate* (IR), then we can compute it as follows.
$$
CAM = PAM \times IR
$$

The PAM estimate can be obtained by multiplying the population density (PD) in the region by the surface of the ROI.

$$
PAM = PD \times \text{surface}(ROI)
$$

In this way, the CAM can be directly computed from the available quantities.

### Definition of ROI
As we said, the simplest region of interest that we can consider is the disk centered at the user's coordinates with a fixed radius, which in this example will be equal to $1$ kilometer.

In [8]:
def surface_roi(r=1000):
    """
    Function that returns the surface of the considered ROI.
    
    For the moment, the considered ROI is a disk of given radius.
    
    Args:
        r: float Radius of the considered disk in meters. (Default: 1000 m)
    Returns:
        float Surface of the ROI.
    """
    return np.pi * r * r

In [17]:
pam = region_density * surface_roi()
cam = pam * old_correct_infected_rate(empirical_infected_rate)
print('Cases around me: ', cam)

Cases around me:  11622.977735553215


In [15]:
cases = []
for ind,i in state_df.iterrows():
    st = i['state']
    pam = i['density']* 1e-6 * surface_roi()
    rate = b[b['name'] == st]['infected_rate'].values[0]
    cases.append(pam * rate)
state_df['cases'] = cases
state_df.head(3)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,state,area,density,cases
0,District of Columbia,158,4251,1758.627463
1,New Jersey,"19,046.80",470,586.809881
2,Rhode Island,"2,678.00",394,140.477692


In [17]:
cases = []
for ind,i in city_df.iterrows():
    st = i['state']
    pam = i['density']* 1e-6 * surface_roi()
    rate = b[b['name'] == st]['infected_rate'].values[0]
    cases.append(pam * rate)
city_df['cases'] = cases
city_df.head(3)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,place,state,area,density,cases
0,Guttenberg,New Jersey,0.508,22015.7,27487.298518
1,Union City,New Jersey,3.320,20045.6,25027.566290
2,West New York,New Jersey,2.610,19059.0,23795.764952


In [18]:
city_df[city_df['place'] == 'New York City']

,place,state,area,density,cases
5,New York City,New York,783.73,10431.1,11686.318582


In [19]:
state_df.sort_values(by='cases', ascending=False)

,state,area,density,cases
0,District of Columbia,158,4251,1758.627463
1,New Jersey,"19,046.80",470,586.809881
7,New York,"122,055.80",162,181.494148
4,Connecticut,"12,540.70",286,158.221365
3,Massachusetts,"20,201.90",336,141.884611
2,Rhode Island,"2,678.00",394,140.477692
18,Michigan,"146,435.30",67,74.370215
5,Maryland,"25,141.00",238,71.729157
6,Delaware,"5,047.90",187,62.537962
17,Georgia,"148,958.00",68,48.132645


In [20]:
city_df

,place,state,area,density,cases
0,Guttenberg,New Jersey,0.508,22015.7,27487.298518
1,Union City,New Jersey,3.320,20045.6,25027.566290
2,West New York,New Jersey,2.610,19059.0,23795.764952
3,Hoboken,New Jersey,3.320,15083.6,18832.352182
4,Kaser,New York,0.440,10729.1,12020.178188
...,...,...,...,...,...
128,Elizabeth,New Jersey,31.910,3916.5,4889.874256
129,Artesia,California,4.200,3903.9,1061.993790
130,Collingdale,Pennsylvania,2.330,3769.2,1278.136394
131,Harwood Heights,Illinois,2.120,3906.7,2038.555998
